In [56]:
import numpy as np
from   scipy.sparse import spdiags, identity
from   scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline  

import inner_task_exp

In [57]:
def psi(t , w, q, mu, T, sigma, r):
   b = q * (r + (mu - r)** 2 / (2* (1 + q) * sigma ** 2))
   
   return np.exp(b * (t - T)) * np.power(w, q) / q

In [58]:
def a_x_t(a_opt_x_t, X, t, W_mesh): # (X -> closest W)! -> a_opt(W, t)
    a = np.zeros_like(X)
    for i, j in enumerate(X):
        if j > np.max(W_mesh):
            a[i] = a_opt_x_t[-1, t]
        elif j < np.min(W_mesh):
            a[i] = a_opt_x_t[0, t]
        else:
            
            a[i] = a_opt_x_t[np.argwhere(W_mesh > j)[0][0] - 1, t]
    return a

In [59]:
def GeneratePathsBSEuler(NoOfPaths, NoOfSteps, T, dt, timesteps, r, W_0, sigma, sigmaxi, hsigsq, a_opt_x_t, W_mesh, eta = 0):  
      
    Z = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps])
    W = np.zeros([NoOfPaths, NoOfSteps+1])
    W_hat = np.zeros([NoOfPaths, NoOfSteps+1])
    Z_hat = np.random.normal(0.0,1.0,[NoOfPaths,NoOfSteps])


    X = np.zeros([NoOfPaths, NoOfSteps+1])
    X[:,0]= W_0
    time = np.zeros_like(timesteps)
        
    for i in range(0,NoOfSteps):

        if NoOfPaths > 1:
            Z[:,i] = (Z[:,i] - np.mean(Z[:,i])) / np.std(Z[:,i])
            Z_hat[:,i] = (Z_hat[:,i] - np.mean(Z_hat[:,i])) / np.std(Z_hat[:,i])
        
        
        W[:,i+1] = W[:,i] + np.power(dt, 0.5)*Z[:,i]
        W_hat[:,i+1] = W_hat[:,i] + np.power(dt, 0.5)*Z_hat[:,i]


        a = a_x_t(a_opt_x_t, X[:, i], i, W_mesh) # make optimal control for every X equal to the one in the closest to X point in W_mesh
        X[:,i+1] = X[:,i] - (( r + a * sigmaxi ) - (hsigsq * (a**2))*dt) - a * sigma*(W[:,i+1]-W[:,i]) - eta * (W_hat[:,i+1]-W_hat[:,i])
        time[i] = i *  dt
     
    paths = {"time":time,"X":X}
    return paths

In [60]:
def f_y2(W, y, q, T, r):
        return (np.power(np.exp(W), q))/q * (y ** 2 + 1)

        
def f_eps_grad(X, eps, y, q, T, r): # make computation of gradient
        return (f_y2(X, y  + eps, q, T, r) - f_y2(X, y, q, T, r)) / eps

In [61]:
r     = 0.03 #parameters of the dynamic
sigma = 0.3
mu    = 0.11

xi    = (mu - r) / sigma    # some helping computations
hsigsq  = 0.5 * sigma ** 2 
sigmaxi = sigma * xi

eps_dyn_W = 0
lam1 = 0
lam2 = 0.1      
alpha = 0.5
q     = 0.1

Wmin  = -60
Wmax  = 60
M     = 500
N     = 1000
T     = 2

tol   = 1e-4
scale = 1.0
A_max  = 1.0
J     = 200


dW      = ( Wmax - Wmin) / N
dt      = T / M
dWsq    = dW ** 2

timesteps = np.linspace( 0.0, T, M + 1 )[:-1] 
timesteps = np.flipud(timesteps)
NoOfSteps = len(timesteps)
NoOfPaths = 100
W_0 = 0
W_mesh = np.linspace( Wmin, Wmax, N + 1 )
eta = 0

y = 1





In [ ]:


lam2 = 0
color_num = 0
colors = ['black', 'b', 'orange', 'pink', 'yellow', 'g']

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,5))

ax1.set_title('Plot of $v(0, x)$ against x'),
ax1.set_xlabel('x')
ax1.set_ylabel('v(0,x)')

ax2.set_title('Plot of $v(0, x)$ against x for x > 48'),
ax2.set_xlabel('x')
ax2.set_ylabel('v(0,x)')

ax3.set_title('Plot of optimal control against x')
ax3.set_xlabel('x')
ax3.set_ylabel('a(0,x)')
ax3.set_ylim(0, 1.5)

for lam2 in [10, 1, 0.1, 0.01, 0.001, 0.0001]:

    y0   = 1
    step = 0.3
    eps_grad = 0.01
    w0   = 0
    V_new = 0
    V_0_w0 = np.inf
    k    = 0
    V_grad = np.inf
    print(lam2)
    while k < 50 and np.abs(step * V_grad) > 0.01:
        print(k)
        V_0_w0 = V_new
        #solving inner task
        W, V, ctrls, VV, opt_c  = inner_task_exp.inner_problem_solver(y0, alpha, Wmin, Wmax, N, A_max, J, q, T, dt, r, timesteps, eps_dyn_W, lam1, lam2, hsigsq, dWsq, sigmaxi)
        #generate paths using optimal control
        path = GeneratePathsBSEuler(NoOfPaths, NoOfSteps, T, dt, timesteps, r, W_0, sigma, sigmaxi, hsigsq, opt_c, W_mesh, eta = 0)
        V_new = V[np.argwhere(W > w0)[0][0] - 1]
        X_T = np.array(path.get('X')[:, -1])
        # find gradient
        V_grad = f_eps_grad(X_T, eps_grad, y0, q, T, r).mean()
        # make gradient step
        y0 = y0 - step * V_grad
        k += 1
        
    y_opt = y0
    W, V, ctrls, VV, opt_c  = inner_task_exp.inner_problem_solver(y0, alpha, Wmin, Wmax, N, A_max, J, q, T, dt, r, timesteps, eps_dyn_W, lam1, lam2, hsigsq, dWsq, sigmaxi)
    V_opt = V[np.argwhere(W > w0)[0][0] - 1]
    print("lam2 = ", lam2, "y0 = ", y0,  V_0_w0, V_grad, k)
    opt_control = xi / ((1+q) * sigma)
    
    ax1.plot(W, V, label=r'$\lambda$=' + f'{lam2}', color = colors[color_num], lw=2)
    ax2.plot(W[900:], V[900:], label=r'$\lambda$=' + f'{lam2}', color = colors[color_num], lw=2)
    ax3.plot(W[20:-20], ctrls[20:-20], label=r'$\lambda$=' + f'{lam2}', color = colors[color_num], lw=2, alpha = 0.7)
    color_num += 1

ax1.plot(W, psi(0, np.exp(W), q, mu, T, sigma, r),'--', label='$\lambda$=0', color = 'red', lw=2)
ax2.plot(W[900:], psi(0, np.exp(W), q, mu, T, sigma, r)[900:],'--', label='$\lambda$=0', color = 'red', lw=2)
ax3.plot(W[20:-20], opt_control * np.ones(W.shape)[20:-20],'--', label=r'$a^{*}$', color = 'red', lw=2, alpha = 0.7)

    
    
ax1.legend()
ax2.legend()
ax3.legend()    

plt.show() 
